In [136]:
import pandas as pd
import numpy as np



Extraer la información del archivo.

In [137]:
df = pd.read_csv("/content/drive/MyDrive/google drive/0.8ddpr74xw650.udno6l076vcountry_vaccinations.csv")

display(df)

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86507,Zimbabwe,ZWE,2022-03-25,8691642.0,4814582.0,3473523.0,139213.0,69579.0,57.59,31.90,23.02,4610.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...",Ministry of Health,https://www.arcgis.com/home/webmap/viewer.html...
86508,Zimbabwe,ZWE,2022-03-26,8791728.0,4886242.0,3487962.0,100086.0,83429.0,58.25,32.38,23.11,5528.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...",Ministry of Health,https://www.arcgis.com/home/webmap/viewer.html...
86509,Zimbabwe,ZWE,2022-03-27,8845039.0,4918147.0,3493763.0,53311.0,90629.0,58.61,32.59,23.15,6005.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...",Ministry of Health,https://www.arcgis.com/home/webmap/viewer.html...
86510,Zimbabwe,ZWE,2022-03-28,8934360.0,4975433.0,3501493.0,89321.0,100614.0,59.20,32.97,23.20,6667.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac...",Ministry of Health,https://www.arcgis.com/home/webmap/viewer.html...


Mostrar la estructura y tipos de datos de cada columna para identificar qué operaciones puedes realizar con cada una de ellas, asegurándote que las columnas con fechas sean del tipo datetime64.


In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86512 entries, 0 to 86511
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              86512 non-null  object 
 1   iso_code                             86512 non-null  object 
 2   date                                 86512 non-null  object 
 3   total_vaccinations                   43607 non-null  float64
 4   people_vaccinated                    41294 non-null  float64
 5   people_fully_vaccinated              38802 non-null  float64
 6   daily_vaccinations_raw               35362 non-null  float64
 7   daily_vaccinations                   86213 non-null  float64
 8   total_vaccinations_per_hundred       43607 non-null  float64
 9   people_vaccinated_per_hundred        41294 non-null  float64
 10  people_fully_vaccinated_per_hundred  38802 non-null  float64
 11  daily_vaccinations_per_milli

La columna date no es de tipo date, por lo que procedere a convertir cambiar el Dtype de "object" a "date"

In [139]:
df['date'] = pd.to_datetime(
    df['date'],
    errors='coerce',
    infer_datetime_format=True
).dt.normalize()


/tmp/ipython-input-3344036835.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date'] = pd.to_datetime(


In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86512 entries, 0 to 86511
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   country                              86512 non-null  object        
 1   iso_code                             86512 non-null  object        
 2   date                                 86512 non-null  datetime64[ns]
 3   total_vaccinations                   43607 non-null  float64       
 4   people_vaccinated                    41294 non-null  float64       
 5   people_fully_vaccinated              38802 non-null  float64       
 6   daily_vaccinations_raw               35362 non-null  float64       
 7   daily_vaccinations                   86213 non-null  float64       
 8   total_vaccinations_per_hundred       43607 non-null  float64       
 9   people_vaccinated_per_hundred        41294 non-null  float64       
 10  people_ful

Determinar la cantidad de vacunas aplicadas de cada compañía (con base en cómo lo reporta cada país en la columna vaccines, en otras palabras, agrupe por vaccines y realice la sumatoria).

In [141]:
df['vaccine_list'] = df['vaccines'].str.split(r',\s*')
df['vaccine_count'] = df['vaccine_list'].str.len()
df['adjusted_vaccinated'] = df['total_vaccinations'] / df['vaccine_count']
df_exploded = df.explode('vaccine_list')
df_exploded['vaccine_list'] = df_exploded['vaccine_list'].str.strip()
vaccine_counts = df_exploded.groupby('vaccine_list')['adjusted_vaccinated'].sum().astype(int).sort_values(ascending=False)
print(vaccine_counts)

vaccine_list
Oxford/AstraZeneca    275640094162
Pfizer/BioNTech       236379325083
Sinovac               211533241395
Sinopharm/Beijing     185512522962
Moderna               177129527846
CanSino               158185869281
Sputnik V             146504369246
Johnson&Johnson       143607377866
Sinopharm/Wuhan       142984340357
ZF2001                142292598716
Covaxin               111379715394
Novavax                26309613253
EpiVacCorona           14194816969
Turkovac               11307700311
Abdala                  5216771883
Soberana02              3419332745
Soberana Plus           2135055191
Sputnik Light           1772999393
Medigen                 1397820225
QazVac                  1197678593
FAKHRAVAC               1188310720
COVIran Barekat         1188310720
Razi Cov Pars           1188310720
SpikoGen                1188310720
Name: adjusted_vaccinated, dtype: int64


Obtener la cantidad de vacunas aplicadas en todo el mundo

In [142]:
total = df['total_vaccinations'].sum().astype(int)
print(total)

2002854013761


Calcular el promedio de vacunas aplicadas por cada pais

In [143]:
df_filtrado = df[df['total_vaccinations'].notna() & (df['total_vaccinations'] > 0)] #importante si no abra datos incongruentes con numeros enteros pequeños con decimales muy grandes
total_pais = df_filtrado.groupby('country')['total_vaccinations'].mean().astype(int).sort_values(ascending=False)
print(total_pais)

country
China               1842734191
India                757471442
United States        329116490
Brazil               183877149
Indonesia            119615402
                       ...    
Falkland Islands          2832
Montserrat                2820
Niue                      2211
Tokelau                   1285
Pitcairn                    69
Name: total_vaccinations, Length: 223, dtype: int64


Determinar la cantidad de vacunas aplicadas el día 29/01/21 en todo el mundo.

In [144]:
filtro_fecha = df[df['date'] == '2021-01-29']

total_vacunas_dia = filtro_fecha['daily_vaccinations'].sum()

print(f"Total de vacunas aplicadas el 29/01/2021: {int(total_vacunas_dia):,}")


Total de vacunas aplicadas el 29/01/2021: 4,884,052


Crear un dataframe nuevo denominado conDiferencias que contenga los datos originales y una columna derivada (diferencias) con las diferencias de aplicación entre las columnas daily_vaccionations y daily_vaccionations_raw.  

In [145]:
conDiferencias = df[['country', 'date', 'daily_vaccinations', 'daily_vaccinations_raw']].copy()
conDiferencias['diferencias'] = conDiferencias['daily_vaccinations'] - conDiferencias['daily_vaccinations_raw']

display(conDiferencias.sample(20))


,country,date,daily_vaccinations,daily_vaccinations_raw,diferencias
18345,Cote d'Ivoire,2021-11-24,32523.0,NaN,NaN
54160,Nauru,2021-12-12,2.0,NaN,NaN
73511,Spain,2021-07-07,556700.0,690749.0,-134049.0
55358,New Caledonia,2021-09-25,3219.0,NaN,NaN
78603,Tonga,2021-11-25,1235.0,NaN,NaN
64565,Russia,2022-03-15,75104.0,69442.0,5662.0
74175,Sri Lanka,2022-03-08,32950.0,29107.0,3843.0
29876,Gibraltar,2021-03-30,730.0,648.0,82.0
19560,Curacao,2022-02-08,206.0,177.0,29.0
7676,Belgium,2021-09-14,13527.0,5141.0,8386.0


Obtener el periodo de tiempo entre el registro con fecha más reciente y el registro con fecha más antigua.

In [146]:
fecha_min = df['date'].min()
fecha_max = df['date'].max()

periodo = fecha_max - fecha_min

print("Fecha más antigua:", fecha_min)
print("Fecha más reciente:", fecha_max)
print("Periodo de tiempo:", periodo)

Fecha más antigua: 2020-12-02 00:00:00
Fecha más reciente: 2022-03-29 00:00:00
Periodo de tiempo: 482 days 00:00:00


Crear un dataframe nuevo denominado conCantidad que contenga los datos originales y una columna derivada (canVac) con la cantidad de vacunas utilizadas cada día (usar la columna vaccines y separar por el carácter , ).

In [147]:
conCantidad = (
    df.groupby('date')['daily_vaccinations']
      .sum()
      .astype(int)
      .reset_index()
)
conCantidad.columns = ['date', 'total_daily_vaccinations']

display(conCantidad.sample(10))


,date,total_daily_vaccinations
98,2021-03-10,9303231
350,2021-11-17,31742052
3,2020-12-05,0
45,2021-01-16,3273953
23,2020-12-25,551048
192,2021-06-12,34227030
312,2021-10-10,25603017
374,2021-12-11,35359710
81,2021-02-21,5966442
273,2021-09-01,41181050


j. Generar un dataframe denominado antes20 con todos los registros que se hayan realizado antes del 20 de diciembre de 2020.

In [148]:
antes20 = df[df['date'] < '2020-12-20']
display(antes20.head())

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website,vaccine_list,vaccine_count,adjusted_vaccinated
13403,Canada,CAN,2020-12-14,5.0,5.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ...",Official data from provinces via covid19tracke...,https://covid19tracker.ca/vaccinationtracker.html,"[Johnson&Johnson, Moderna, Oxford/AstraZeneca,...",4,1.25
13404,Canada,CAN,2020-12-15,727.0,727.0,NaN,722.0,722.0,0.00,0.00,NaN,19.0,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ...",Official data from provinces via covid19tracke...,https://covid19tracker.ca/vaccinationtracker.html,"[Johnson&Johnson, Moderna, Oxford/AstraZeneca,...",4,181.75
13405,Canada,CAN,2020-12-16,3025.0,3025.0,NaN,2298.0,1510.0,0.01,0.01,NaN,40.0,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ...",Official data from provinces via covid19tracke...,https://covid19tracker.ca/vaccinationtracker.html,"[Johnson&Johnson, Moderna, Oxford/AstraZeneca,...",4,756.25
13406,Canada,CAN,2020-12-17,7279.0,7279.0,NaN,4254.0,2425.0,0.02,0.02,NaN,64.0,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ...",Official data from provinces via covid19tracke...,https://covid19tracker.ca/vaccinationtracker.html,"[Johnson&Johnson, Moderna, Oxford/AstraZeneca,...",4,1819.75
13407,Canada,CAN,2020-12-18,11296.0,11296.0,NaN,4017.0,2823.0,0.03,0.03,NaN,74.0,"Johnson&Johnson, Moderna, Oxford/AstraZeneca, ...",Official data from provinces via covid19tracke...,https://covid19tracker.ca/vaccinationtracker.html,"[Johnson&Johnson, Moderna, Oxford/AstraZeneca,...",4,2824.00


 Obtener un dataframe denominado pfizer con todos los registros donde se haya utilizado la vacuna Pfizer.

In [149]:
df['vaccine_list'] = df['vaccines'].str.split(r',\s*')
df['vaccine_count'] = df['vaccine_list'].str.len()
df['adjusted_vaccinated'] = df['total_vaccinations'] / df['vaccine_count']
df_exploded = df.explode('vaccine_list')
df_exploded['vaccine_list'] = df_exploded['vaccine_list'].str.strip()
pfizer_only = df_exploded[df_exploded['vaccine_list'] == 'Pfizer/BioNTech']
pfizer_total = pfizer_only['adjusted_vaccinated'].sum().astype(int)
pfizer = pd.DataFrame({
    'vaccine': ['Pfizer/BioNTech'],
    'total_adjusted_vaccinated': [pfizer_total]
})

display(pfizer)


,vaccine,total_adjusted_vaccinated
0,Pfizer/BioNTech,236379325083


Creacion del excel con los dataframe previamente elaborados, donde cada df es una hoja diferente del excel

In [150]:
#!pip install xlsxwriter

In [151]:
with pd.ExcelWriter('resultadosReto.xlsx', engine='xlsxwriter') as writer:
  conDiferencias.to_excel(writer, sheet_name = 'Diferencias', index = False)
  conCantidad.to_excel(writer, sheet_name = 'Cantidad', index = False)
  antes20.to_excel(writer, sheet_name = 'Antes20', index = False)
  pfizer.to_excel(writer, sheet_name = 'Pfizer', index = False)